# Project - Airline AI Assistant
I've added database connectivity to enable Openai to:
- Retrieve ticket prices
- Display the number of available seats for each flight
- List all available destination cities
- Facilitate seat bookings

Once a booking is confirmed, an image of the booked destination city is displayed.

In [0]:
# imports
import os
import json
import gradio as gr
import mysql.connector
import base64
from dotenv import load_dotenv
from openai import OpenAI
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play
from PIL import Image

In [0]:
# Initialization
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [0]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "Make sure you ask if they want to book a flight when appropriate."
system_message += "If they book a flight make sure you respond with 'Booking confirmed' in your reply."

In [0]:
def get_db_connection():
    return mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME")
    )

In [0]:

def get_ticket_price(destination_city):
    db_connection = get_db_connection()
    cursor = db_connection.cursor()
    select_query = "SELECT price FROM flights WHERE z_city = %s;"
    cursor.execute(select_query, (destination_city,))
    # print(f"QUERY: {select_query}")
    row = cursor.fetchone()
    cursor.close()
    db_connection.close()

    return float(row[0]) if row else None

In [0]:
def get_avail_seats(destination_city):
    db_connection = get_db_connection()
    cursor = db_connection.cursor()
    select_query = """
        SELECT f.seats - COALESCE(b.booked, 0) AS available
        FROM flights f
        LEFT JOIN (
            SELECT flight_number, COUNT(*) AS booked
            FROM bookings
            GROUP BY flight_number
        ) b ON f.flight_number = b.flight_number
        WHERE f.z_city = %s;
    """
    cursor.execute(select_query, (destination_city,))
    row = cursor.fetchone()

    cursor.close()
    db_connection.close()

    return row[0] if row else None

In [0]:
def book_seat(destination_city, passenger):
    db_connection = get_db_connection()
    cursor = db_connection.cursor()

    cursor.execute("SELECT flight_number FROM flights WHERE z_city = %s LIMIT 1;", (destination_city,))
    flight = cursor.fetchone()

    if not flight:
        cursor.close()
        db_connection.close()
        return {"error": f"No available flights to {destination_city}."}

    flight_number = flight[0]  # Extract the flight number from the result

    insert_query = "INSERT INTO bookings (`name`, `flight_number`) VALUES (%s, %s);"
    cursor.execute(insert_query, (passenger, flight_number))
    db_connection.commit()

    confirmation = {
        "message": f"Booking confirmed for {passenger} to {destination_city}.",
        "flight_number": flight_number
    }

    cursor.close()
    db_connection.close()
    
    return confirmation

In [0]:
def get_destinations():
    db_connection = get_db_connection()
    cursor = db_connection.cursor()
    
    select_query = "SELECT DISTINCT z_city FROM flights;"  # Ensure unique destinations
    cursor.execute(select_query)
    rows = cursor.fetchall()  # Fetch all rows
    destinations = [row[0] for row in rows] if rows else []  # Extract city names
    cursor.close()
    db_connection.close()
    
    return destinations  # Returns a list of destination cities

In [0]:
tool_call = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the price of a return ticket to the destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to"
                    }
                },
                "required": ["destination_city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_avail_seats",
            "description": "Get the number of available seats to the destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to"
                    }
                },
                "required": ["destination_city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_destinations",
            "description": "Fetches available flight destinations (city pairs) and their corresponding prices.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_seat",
            "description": "Book seat to the destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to"
                    },
                    "passenger": {
                        "type": "string",
                        "description": "The passenger booking the flight"
                    }
                },
                "required": ["destination_city","passenger"]
            }
        }
    }
]

In [0]:
def handle_tool_call(message):
    if not message.tool_calls:
        raise ValueError("No tool calls found in the message.")

    tool_call = message.tool_calls[0]  
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    function_name = tool_call.function.name

    # Handle function calls
    if function_name == "get_ticket_price":
        reply = get_ticket_price(city)
        key = "price"
    elif function_name == "get_avail_seats":
        reply = get_avail_seats(city)
        key = "seats"
    elif function_name == "get_destinations":
        reply = get_destinations()
        key = "destinations"
    elif function_name == "book_seat":
        passenger = arguments.get("passenger")  # Extract passenger name
        if not passenger:
            raise ValueError("Passenger name is required for booking.")
        reply = book_seat(city, passenger)
        key = "booking"
    else:
        raise ValueError(f"Unknown function: {function_name}")

    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, key: reply}),
        "tool_call_id": tool_call.id
    }

    return response, city

In [0]:
def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [0]:
def draw_city(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [0]:
def chat(message, history):
    history.append({"role": "user", "content": message})
    messages = [{"role": "system", "content": system_message}] + history
    # print(f"BEFORE TOOL CALL: {message} \n")
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tool_call)
    image = None
    city = None
    
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        response, city = handle_tool_call(tool_message)
        messages.append(tool_message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        talker(response.choices[0].message.content) 
    
    if "Booking confirmed" in response.choices[0].message.content and city:
        image = draw_city(city)

    new_message = response.choices[0].message.content
    history.append({"role": "assistant", "content": new_message})

    return "", history, image

In [0]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=600)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    entry.submit(chat, inputs=[entry, chatbot], outputs=[entry, chatbot, image_output])
    clear.click(lambda: ([], None), inputs=None, outputs=[chatbot, image_output], queue=False)

ui.launch(inbrowser=False)